In [1]:
import os
import torch
import logging
import argparse
import math
import numpy as np
from tqdm import tqdm
import multiprocessing
import time
from utils import convert_examples_to_features, read_review_examples
import json
from torch.utils.data import Dataset
from itertools import cycle
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from models import build_or_load_gen_model
from configs import add_args, set_seed, set_dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os
from transformers import T5Tokenizer, T5Model
from utils import read_review_examples, convert_examples_to_features, TextDataset
from models import build_or_load_gen_model
from run_pre_training import get_loaders
import multiprocessing
import torch.distributed as dist

In [2]:
class FakeArgs(object):
    def __init__(self):
        self.max_source_length = 512
        self.max_target_length = 256
        self.model_type = "t5"
        self.config_name = "t5-base"
        self.add_lang_ids = True
        self.tokenizer_path = "Salesforce/codet5-base"
        self.model_name_or_path = "t5-base"
        self.load_model_path = None
        self.train_path = "../../../lzzz/processed"
        self.train_batch_size = 1
        self.langs = ["ruby"]
        self.cpu_count = 3
        self.num_train_epochs = 2
        self.gradient_accumulation_steps = 1
        self.mask_rate = 0.15
        self.from_scratch = True

args = FakeArgs()
config, model, tokenizer = build_or_load_gen_model(args)
pool = multiprocessing.Pool(args.cpu_count)
datafiles = ["../../../lzzz/processed/ruby_gen.jsonl"]
packs = get_loaders(datafiles, args, tokenizer, pool)
for pack in packs:
    dataset, _, loader = pack
    data = next(loader)
    break

12/24/2021 13:38:01 - INFO - models -   Finish loading model [223M] from t5-base


In [3]:
examples = data
source_ids = torch.tensor(
    [ex.source_ids for ex in examples], dtype=torch.long
)
source_labels = torch.tensor(
    [ex.source_labels for ex in examples], dtype=torch.long
)
target_ids = torch.tensor(
    [ex.target_ids for ex in examples], dtype=torch.long
)
source_mask = source_ids.ne(tokenizer.pad_id)
target_mask = target_ids.ne(tokenizer.pad_id)


In [4]:
model(
    input_ids=source_ids,
    input_labels=source_labels,
    decoder_input_ids=target_ids,
    attention_mask=source_mask,
    decoder_attention_mask=target_mask,
)

torch.FloatTensor
torch.LongTensor


RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
ids = tokenizer.encode("testo")
tokenizer.convert_ids_to_tokens(ids)

In [ ]:
inputs = data[0].source_ids
labels = data[0].source_labels
targets = data[0].target_ids
# print("".join())

In [ ]:
def prettyprint(inputs):
    for i, tok in enumerate(inputs):
        if i > 0 and str(tok).startswith("<e"):
            print("")
        print(str(tok) + " ", end="")
toks = tokenizer.convert_ids_to_tokens(inputs)
labs = labels
tgts = tokenizer.convert_ids_to_tokens(targets)
# print(toks)
prettyprint(toks)
print("\n\n\n\n\n")
prettyprint(labs)
print("\n\n\n\n\n")
prettyprint(tgts)
print("\n\n\n\n\n")


In [ ]:
print("\n".join(dataset.examples[0].prevlines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].lines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].afterlines))
print("\n\n\n\n\n")
print(dataset.examples[0].msg)

In [ ]:
import json
data = []
idx = 0
with open("../../../lzzz/processed/ruby_cls.jsonl", "r") as f:
    for line in f:
        js = json.loads(line)
        data.append(js)
        # print(js["patch"])
        idx += 1
        if idx == 100:
            break